In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as pyplt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import RobustScaler, PowerTransformer

In [ ]:
og_df = pd.read_csv('Data/healthcare-dataset-stroke-data.csv')
og_df.head(3)

In [ ]:
og_df.info()

In [ ]:
og_df.isna().sum()

In [ ]:
for col in og_df:
    if og_df[col].nunique() < 10:
        print(og_df[col].value_counts())
        print("\n")

In [ ]:
#drop id column due to its irrelevance regarding stroke prediction
df_1 = og_df.drop("id", axis=1)
df_1.head(3)

In [ ]:
index = df_1.index[df_1["gender"] == "Other"]
print(df_1.loc[index])

#removed the other gender since there is only one entry, leading to unnecessary complexity
df_2 = df_1.drop(index)

In [ ]:
#replaced NaN values in the bmi column with the average bmi
average_bmi = df_2["bmi"].mean()
df_2["bmi"].fillna(average_bmi, inplace=True)

In [ ]:
# Defined the age bins
bins = range(0, int(df_2["age"].max()) + 10, 10)

# Add a new column "age_group" to the DataFrame representing the age group of each person
df_2["age_group"] = pd.cut(df_2["age"], bins, right=False)

# Group by "age_group" and "gender" and get the size of each group
grouped = df_2.groupby(["age_group", "gender"]).size().unstack()


# Plot the pyramid chart
fig, ax = pyplt.subplots()

ax.barh(grouped.index.astype(str), grouped["Male"], color='lightblue', label="Male")
ax.barh(grouped.index.astype(str), -grouped["Female"], color='lightpink', label="Female")

ax.legend()
ax.minorticks_on()

ax.grid(True, axis="x", color="lightgray", which="both")

pyplt.xlabel("Count")
pyplt.ylabel("Age")

pyplt.show()

In [ ]:
# Defined the age bins
bins = range(0, int(df_2["age"].max()) + 10, 10)

# Add a new column "age_group" to the DataFrame representing the age group of each person
df_2["age_group"] = pd.cut(df_2["age"], bins, right=False)

# Group by "age_group" and "gender" and "stroke" and get the size of each group
grouped = df_2.groupby(["age_group", "gender", "stroke"]).size().unstack(level=[1, 2])

#create pyramid chart
fig, ax = pyplt.subplots(figsize=(12, 8))

#create bar for ech group
for stroke_status in [0, 1]:
    ax.barh(
        grouped.index.astype(str), 
        grouped[("Male", stroke_status)], 
        color='lightblue' if stroke_status == 0 else 'blue', 
        label=f"Male (Stroke: {stroke_status})",
        left=grouped[("Male", 0)] if stroke_status == 1 else 0
    )
    ax.barh(
        grouped.index.astype(str), 
        -grouped[("Female", stroke_status)], 
        color='lightpink' if stroke_status == 0 else 'red', 
        label=f"Female (Stroke: {stroke_status})",
        left=-grouped[("Female", 0)] if stroke_status == 1 else 0
    )


#plot of chart
ax.legend()
ax.minorticks_on()
ax.grid(True, axis="x", color="lightgray", which="both")
pyplt.xlabel("Count")
pyplt.ylabel("Age Group")
pyplt.show()


In [ ]:
# labels
df_2['stroke_label'] = df_2['stroke'].map({0: 'No Stroke', 1: 'Stroke'})

# Age distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.boxplot(data=df_2, x='stroke_label', y='age')
pyplt.title('Age Distribution by Stroke Status')
pyplt.xlabel('Stroke Status')
pyplt.ylabel('Age')
pyplt.show()

# Average glucose level distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.boxplot(data=df_2, x='stroke_label', y='avg_glucose_level')
pyplt.title('Average Glucose Level Distribution by Stroke Status')
pyplt.xlabel('Stroke Status')
pyplt.ylabel('Average Glucose Level')
pyplt.show()

# BMI distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.boxplot(data=df_2, x='stroke_label', y='bmi')
pyplt.title('BMI Distribution by Stroke Status')
pyplt.xlabel('Stroke Status')
pyplt.ylabel('BMI')
pyplt.show()


In [ ]:
#labels
df_2['stroke_label'] = df_2['stroke'].map({0: 'No Stroke', 1: 'Stroke'})

# Gender distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='gender', hue='stroke_label')
pyplt.title('Gender Distribution by Stroke Status')
pyplt.legend(title='Stroke Status')
pyplt.show()

# Hypertension distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='hypertension', hue='stroke_label')
pyplt.title('Hypertension Distribution by Stroke Status')
pyplt.legend(title='Stroke Status')
pyplt.show()

# Heart disease distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='heart_disease', hue='stroke_label')
pyplt.title('Heart Disease Distribution by Stroke Status')
pyplt.legend(title='Stroke Status')
pyplt.show()

# Ever married distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='ever_married', hue='stroke_label')
pyplt.title('Ever Married Distribution by Stroke Status')
pyplt.legend(title='Stroke Status')
pyplt.show()

# Work type distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='work_type', hue='stroke_label')
pyplt.title('Work Type Distribution by Stroke Status')
pyplt.xticks(rotation=45)
pyplt.legend(title='Stroke Status')
pyplt.show()

# Residence type distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='Residence_type', hue='stroke_label')
pyplt.title('Residence Type Distribution by Stroke Status')
pyplt.legend(title='Stroke Status')
pyplt.show()

# Smoking status distribution by stroke status
pyplt.figure(figsize=(6, 3))
sns.countplot(data=df_2, x='smoking_status', hue='stroke_label')
pyplt.title('Smoking Status Distribution by Stroke Status')
pyplt.xticks(rotation=45)
pyplt.legend(title='Stroke Status')
pyplt.show()


In [ ]:

fig, ax = pyplt.subplots(figsize=(12, 8))

#create boxplot
df_2.boxplot(column='age', by=['stroke', 'gender'], ax=ax, grid=False, patch_artist=True, 
             boxprops=dict(facecolor='lightblue', color='blue'),
             medianprops=dict(color='red'),
             whiskerprops=dict(color='blue'),
             capprops=dict(color='blue'),
             flierprops=dict(marker='o', color='blue', alpha=0.5))

#add descriptions
ax.set_title('Age Distribution by Stroke Status and Gender')
ax.set_xlabel('Group')
ax.set_ylabel('Age')
pyplt.suptitle('')

#add x-axis
group_labels = ['No Stroke\nMale', 'No Stroke\nFemale', 'Stroke\nMale', 'Stroke\nFemale']
ax.set_xticklabels(group_labels)

#plot chart
pyplt.show()

In [ ]:
#select only numerical columns to create a heatmap
num_df = df_2.select_dtypes(include=['int64', 'float64'])

#calculate the correlation matrix
corr = num_df.corr()

#plot the heatmap
sns.heatmap(corr, annot=True, mask = np.triu(np.ones_like(corr, dtype=bool)))

In [ ]:
#select only non-numerical columns to create dummies
non_num_df = df_2.select_dtypes(exclude=['int64', 'float64'])

#convert non-numerical columns to dummies
dummies_df = pd.get_dummies(non_num_df)

#convert dummies to integers
dummies_df = dummies_df.astype(int)

#concatenate the numerical and dummies dataframes
completeNum_df = pd.concat([num_df, dummies_df], axis=1)

#calculate the correlation matrix
completeCorr = completeNum_df.corr()

#plot the heatmap
pyplt.figure(figsize=(25, 25))
sns.heatmap(completeCorr, annot=True, mask = np.triu(np.ones_like(completeCorr, dtype=bool)))


In [ ]:
stroke_corr = completeCorr['stroke']

# Define a threshold for strong correlation
threshold = 0.05

# Select values where the absolute correlation with 'stroke' is above the threshold
strong_corr = stroke_corr[stroke_corr > threshold]
strong_corr.sort_values(ascending=False, inplace=True)

# Select values where the absolute correlation with 'stroke' is below the threshold
weak_corr = stroke_corr[stroke_corr < threshold]
weak_corr.sort_values(ascending=False, inplace=True)

print(strong_corr)
print("\n")
print(weak_corr)

In [ ]:
z_scores = np.abs(stats.zscore(num_df))

threshold = 3

outliers = num_df[(z_scores > threshold).any(axis=1)]

print(outliers)

### Medizinische Risikofaktoren 

Aus dem Auszug aus "Der Nervenarzt" (Weih, M., Müller-Nordhorn, J., Amberger, N., Masuhr, F., Lürtzing, F., Dreier, J. P., & Hetzel, A. (2004). Risikofaktoren des ischämischen Schlaganfalls. Der Nervenarzt, 75(4), 324-335.) gehen folgende Risikofaktoren für einen Schlaganfall hervor:


[Hier ist das Dokument verlinkt](https://d1wqtxts1xzle7.cloudfront.net/94400450/s00115-003-1639-620221117-1-1gmosza-libre.pdf?1668693481=&response-content-disposition=inline%3B+filename%3DRisikofaktoren_des_isch_mischen_Schlagan.pdf&Expires=1717179362&Signature=bGb58O9nASWfgCQqpNkQFhmIHglmKAvrY3kmiZBI3tDAi1DlG~VZKiIfktxvCrA0xkuwOhSPU06fMT0U7xKca0pYi2KamjsU3aM9pTJE-jotX7kzQ3rEVYaYzT-BdPSRK2uY4-j2-jQSbrxPOgs0P53mV1YWwopvOcYulQYx0caoDckv1LegUKDGacm6Fd2s0bmvyJO7-11wJOvM2iasaOvPofmJhggWYMWB~~zC4JHON4Kz8XYNrcW3g2ywKfwVX0U2NgCOxHdJ26YFsaLNm~ebMgrEHipW5WF-VFHvR89ro605zx0J5UFukRoP3beZ2abSw8z2lnFyoF9MKRLPzg__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA)



##### Risikofaktoren für Schlaganfälle: (S.334 f)

1. **Hypertonie**: Aufgrund ihrer hohen Prävalenz und ihres starken Einflusses auf das Schlaganfallrisiko ist Hypertonie einer der bedeutendsten Risikofaktoren.


2. **Vorhofflimmern**: Die Antikoagulation bei Vorhofflimmern kann das Schlaganfallrisiko signifikant reduzieren.


3. **Atherosklerose-bezogene Risikofaktoren**: Dazu gehören hohe Cholesterinwerte, die vor allem für atherothrombotische Infarkte relevant sind. Statine können das Schlaganfallrisiko bei Patienten mit KHK oder Hypercholesterinämie reduzieren.


4. **Verhaltensabhängige Lebensstilfaktoren**: Rauchen, Übergewicht, körperliche Inaktivität und ungesunde Ernährung sind ebenfalls wichtige Risikofaktoren, die durch Interventionen modifiziert werden können.


5. **Kardiale Risikofaktoren**: Dazu gehören persistierendes Foramen ovale, Herzinsuffizienz und koronare Herzkrankheit, die das Schlaganfallrisiko erhöhen können.


6. **Gerinnungsstörungen**: Obwohl das Evidenzniveau und das Verständnis dieser Risikofaktoren heterogen sind, können Gerinnungsstörungen wie APC-Resistenz, Antiphospholipid-Antikörper, Hyperhomozysteinämie und Hyperfibrinogenämie das Risiko erhöhen. 


7. **Weitere Risikofaktoren**: Dazu gehören Migräne, orale Kontrazeptiva, Entzündungen (z.B. durch CRP) und möglicherweise Infektionen (z.B. durch Chlamydien oder Helicobacter).

In [ ]:
stroke_yes = df_2[df_2['stroke'] == 1]
stroke_no = df_2[df_2['stroke'] == 0]

fig, ax = pyplt.subplots(nrows=2, ncols=3, figsize=(25, 12))	

sns.boxplot(x=df_2['bmi'], ax=ax[0,0])
ax[0,0].set_title('Stroke: Both')

sns.boxplot(x=stroke_yes['bmi'], ax=ax[0,1])
ax[0,1].set_title('Stroke: Yes')

sns.boxplot(x=stroke_no['bmi'], ax=ax[0,2])
ax[0,2].set_title('Stroke: No')

sns.boxplot(x=df_2['avg_glucose_level'], ax=ax[1,0])
ax[1,0].set_title('Stroke: Both')

sns.boxplot(x=stroke_yes['avg_glucose_level'], ax=ax[1,1])
ax[1,1].set_title('Stroke: Yes')

sns.boxplot(x=stroke_no['avg_glucose_level'], ax=ax[1,2])
ax[1,2].set_title('Stroke: No')

In [ ]:
df_nml = df_2.drop(["age_group", "stroke_label"], axis=1)	

pt = PowerTransformer(method='box-cox', standardize=False, copy=False)

# Transformation der Daten
df_nml[['bmi', 'avg_glucose_level']] = pt.fit_transform(df_nml[['bmi', 'avg_glucose_level']])

df_nml

stroke_yes = df_nml[df_nml['stroke'] == 1]
stroke_no = df_nml[df_nml['stroke'] == 0]

fig, ax = pyplt.subplots(nrows=2, ncols=3, figsize=(25, 12))	

sns.boxplot(x=df_nml['bmi'], ax=ax[0,0])
ax[0,0].set_title('Stroke: Both')

sns.boxplot(x=stroke_yes['bmi'], ax=ax[0,1])
ax[0,1].set_title('Stroke: Yes')

sns.boxplot(x=stroke_no['bmi'], ax=ax[0,2])
ax[0,2].set_title('Stroke: No')

sns.boxplot(x=df_nml['avg_glucose_level'], ax=ax[1,0])
ax[1,0].set_title('Stroke: Both')

sns.boxplot(x=stroke_yes['avg_glucose_level'], ax=ax[1,1])
ax[1,1].set_title('Stroke: Yes')

sns.boxplot(x=stroke_no['avg_glucose_level'], ax=ax[1,2])
ax[1,2].set_title('Stroke: No')

<h3>4Balancing<h3>

In [ ]:

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


In [ ]:
df_nml.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
l3 = []; l4 = [];
print('Label Encoder Transformation')
for i in categorical_cols:
    df_nml[i] = le.fit_transform(df_nml[i])
    l3.append(list(df_nml[i].unique())); l4.append(list(le.inverse_transform(df_nml[i].unique())))
    print(i,' : ',df_nml[i].unique(),' = ',le.inverse_transform(df_nml[i].unique()))

In [ ]:
df_nml.head()   

In [ ]:
features = df_nml.drop("stroke", axis=1)
target = df_nml["stroke"]

under_sampler = RandomUnderSampler(sampling_strategy=0.075)
smote = SMOTE(sampling_strategy=1)


features_under, target_under = under_sampler.fit_resample(features, target)

features_balanced, target_balanced = smote.fit_resample(features_under, target_under)

Counter(target_balanced)



In [ ]:
features_balanced.to_csv('Data/Features.csv', index=False)

In [ ]:
one change